In [1]:
'''
Author: Zhen Liu lzhen.dev@outlook.com
CreateDate: Do not edit
LastEditors: Zhen Liu lzhen.dev@outlook.com
LastEditTime: 2024-03-02
Description: 

Copyright (c) 2024 by HernandoR lzhen.dev@outlook.com, All Rights Reserved. 
'''
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import random
import numpy as np
import pandas as pd
import wandb

from typing import Callable
from datetime import datetime

import torch

from pytorch_tabular.utils import make_mixed_dataset, print_metrics, load_covertype_dataset
from pytorch_tabular.utils import get_balanced_sampler, get_class_weighted_cross_entropy

import os
%load_ext autoreload
%autoreload 2

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hernando (lzhen-ntu). Use `wandb login --relogin` to force relogin


True

In [3]:
# data, cat_col_names, num_col_names = make_mixed_dataset(
#     task="classification", 
#     n_samples=10000, 
#     n_features=8, 
#     n_categories=4, 
#     weights=[0.8], 
#     random_state=42)
# target_col='target'


In [4]:

Feature_Explain=pd.read_excel('data/Amex_ori/Amex Campus.xlsx')
Feature_target_cols = Feature_Explain['Feature Name'][Feature_Explain['Extended description']=='Target'].to_list()
Feature_cat_col_names = Feature_Explain['Feature Name'][Feature_Explain['Feature Type']=='categorical'].to_list()
Feature_num_col_names = Feature_Explain['Feature Name'][Feature_Explain['Variable Type'] =='numeric'].to_list()

DEVICE="mps" if torch.backends.mps.is_available() else ('cuda' if torch.backends.cuda.is_available() else "cpu")


In [24]:
data = pd.read_parquet('data/train.parquet')

cols=data.columns.to_list()
target_cols = [col for col in cols if col in Feature_target_cols]
if len(target_cols) == 0:
    print('No target column found')
    exit()
elif len(target_cols) > 1:
    target_col=target_cols[0]
cat_col_names = [col for col in cols if col in Feature_cat_col_names]
num_col_names = [col for col in cols if col in Feature_num_col_names]

print(f"{target_cols}")
# target_col=cols.unite(Feature_target_cols)

# target_col = 'activation'

['activation', 'ind_recommended']


In [25]:

data.head()
print(f"target_col={target_col}, cat_col_names={cat_col_names}, num_col_names={num_col_names}")

target_col=activation, cat_col_names=['merchant_profile_01'], num_col_names=['customer_digital_activity_02', 'customer_profile_01', 'customer_profile_02', 'customer_profile_03', 'customer_profile_04', 'customer_spend_06', 'customer_spend_07', 'distance_05']


In [26]:


train, test = train_test_split(data, random_state=42)
train, val = train_test_split(train, random_state=42)

In [27]:
# data['customer_digital_activity_01'].value_counts(normalize=True)
# data.describe()
# len(train)

# Importing the Library

In [28]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

In [29]:
results = []

## Define the Configs


In [30]:
data_config = DataConfig(
    target=[target_col], 
    #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    num_workers=10
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=5210,
    max_epochs=100,
    early_stopping="valid_loss", # Monitor valid_loss for early stopping
    early_stopping_mode = "min", # Set the mode as min because for val_loss, lower is better
    early_stopping_patience=5, # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss", # Save best checkpoint monitoring val_loss
    checkpoints_path = 'checkpoints', # Save the checkpoint in the experiment directory
    checkpoints_save_top_k = 5,
    progress_bar='simple',
    load_best=True, # After training, load the best checkpoint
    # accelerator=DEVICE,
    trainer_kwargs={
        "enable_progress_bar":True, 
        "max_steps":1000,
        # "progress_bar_refresh_rate":1
    }
)
optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="", # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming"
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

EXP_PROJECT_NAME = "AMEX"


In [31]:

model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="1024-512-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
    learning_rate = 1e-3,
    metrics=["f1_score","accuracy"], 
    metrics_params=[{"num_classes":2},{}], # f1_score needs num_classes
    metrics_prob_input=[True, False] # f1_score needs probability scores, while accuracy doesn't
)

experiment_config = ExperimentConfig(
    project_name=EXP_PROJECT_NAME,
    run_name=f"{target_col}",
    exp_watch="gradients",
    # log_target="wandb", # wandb will raise error, too many indicat for 1-d data.
    log_logits=True
)



## Training the Model 

In [32]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    experiment_config=experiment_config,
    verbose=False,
    # suppress_lightning_logger=True,

)

In [33]:
sampler = get_balanced_sampler(train[target_col].values.ravel())

tabular_model.fit(train=train, validation=val, train_sampler=sampler)
# takes ~ 60min,13 Epoch, most time in preparing data, weird

Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniforge/base/envs/AMEX/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /Users/lz/Codes/AE2024/checkpoints exists and is not empty.
/opt/homebrew/Caskroom/miniforge/base/envs/AMEX/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
/opt/homebrew/Caskroom/miniforge/base/envs/AMEX/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Trainer was signaled to stop but the required `min_epochs=1` or `min_steps=None` has not been met. Training will continue...
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.0001
Restoring states from the checkpoint path at /Users/lz/Codes/AE2024/.lr_find_9c8f10ef-3028-4a3e-8014-c75038d1b07e.ckpt
Restored all states from the checkpoint at /Users/lz/Codes/AE2024/.lr_find_9c8f10ef-3028-4a3e-8014-c75038d1b07e.ckpt

  | Name             | Type                      | Params
---------------------------------------------------------------
0 | _backbone        | CategoryEmbeddingBackbone | 833 K 
1 | _embedding_layer | Embedding1dLayer          | 2.6 K 
2 | head             | LinearHead                | 1.0 K 
3 | loss             | CrossEntropyLoss          | 0     
---------------------------------------------------------------
837 K     Trainable params
0         Non-trainable params
837 K     Total params
3.349     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


In [34]:
result = tabular_model.evaluate(test)
# pred_df = tabular_model.predict(test)
tabular_model.save_model(f"{trainer_config.checkpoints_path}/{target_col}") # save in the dir


/opt/homebrew/Caskroom/miniforge/base/envs/AMEX/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7198559641838074     │
│       test_f1_score       │    0.7198559641838074     │
│         test_loss         │    0.5192237496376038     │
└───────────────────────────┴───────────────────────────┘

2024-03-06 14:38:46,310 - {pytorch_tabular.tabular_model:1533} - WARNING - Directory is not empty. Overwriting the 
contents.

In [35]:
# pred_df.head()

In [41]:
def extractSubmision(input_df:pd.DataFrame,
                     pred_col: str,
                     cm_key='customer',
                     mct_key='merchant',
                     output_path='submission.csv')->None:
    '''
    function that extracts the submission file for the AMEX Singapore Hackathon 2024

    input_df : pandas Dataframe which has customer, merchant and pred_col
    pred_col : name of your prediction score variable
    cm_key : customer unique ID (do not change)
    mct_key : merchant unique ID (do not change)
    output_path : path to save the submission file

    Returns - None
    '''
    print('formate prodiction')
    LEN=len(input_df)
    if LEN!=12604600:
        raise f"miss matchlenth {LEN} "
    output_df=pd.DataFrame()
    output_df[[cm_key,mct_key,'predicted_score']]=input_df[[cm_key,mct_key,pred_col]].apply(pd.to_numeric,errors='coerce')
    output_df.to_csv(output_path,index=False)
    return None

def calc(rec:int,act:int):
    crit=rec*2+act
    match crit:
        case 3: # TT
            return 3
        case 0: # NN
            return 2
        case 2: # TN
            return 1
        case 1: # NT
            return 0
    return None
        
# add colum pre_score by func calc eat 'ind_recommended' and 'activaton'
        
def loadmodel_and_predict(modelpath:str,test_data:pd.DataFrame,data_preprocess:Callable = None) -> pd.DataFrame:
    print(f"loading model at {modelpath}")
    loadedmodel=TabularModel.load_model(modelpath)
    # idf=pd.read_csv(test_path)
    if data_preprocess:
        test_data=data_preprocess(test_data)
    print(f"predicting")
    return loadedmodel.predict(test_data)['prediction']

def gen_submission(pred_df:pd.DataFrame):
    print("generating_pred_score")
    for pred in ["ind_recommended","activation",'customer','merchant']:
        if pred not in pred_df.columns:
            print(f"missing col {pred}")


    pred_df['predicted_score']=pred_df.apply(lambda x:calc(x['ind_recommended'],x['activation']),axis=1)

    extractSubmision(pred_df,'predicted_score')
    return None

def preprocess(idf: pd.DataFrame):
    cols=[]
    for col in idf.columns:
        if (col not in cat_col_names) and (col not in num_col_names):
            cols.append(col)
    idf.drop(columns=cols)
    return idf


In [42]:

idf=pd.read_csv('data/Amex_ori/Amex Campus Challenge Round 1.csv')
idf.head()

,distance_05,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,customer_spend_03,...,merchant_profile_02,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,customer,merchant
0,1.621171,NaN,112.3340,80.5525,9.0,966.63,12.0,10.0,4.0,41.0,...,0.437500,26299.0,4777.0,0.8,33.30,72.268283,7.0,423.0,467915,599167
1,2.441944,NaN,112.3340,NaN,NaN,NaN,NaN,NaN,4.0,41.0,...,0.397059,7122.0,4803.0,0.8,793.29,72.268283,7.0,423.0,467915,686617
2,2.438082,NaN,112.3340,71.1925,3.0,284.77,4.0,4.0,4.0,41.0,...,NaN,7222.0,14860.0,0.8,100.00,72.268283,7.0,423.0,467915,829193
3,2.072182,NaN,112.3340,NaN,NaN,NaN,NaN,NaN,4.0,41.0,...,0.142857,11410.0,11968.0,0.8,252.38,72.268283,7.0,423.0,467915,1077034
4,2.380853,NaN,302.7925,NaN,NaN,NaN,NaN,NaN,3.0,37.0,...,0.100000,1847.0,5842.0,0.8,87.50,72.268283,7.0,423.0,467915,876647


In [44]:

for target in target_cols:
    idf[target]=loadmodel_and_predict(f"{trainer_config.checkpoints_path}/{target}",idf,preprocess)


loading model at checkpoints/activation


2024-03-06 14:53:09,808 - {pytorch_tabular.tabular_model:340} - INFO - Preparing the Trainer

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


predicting


Output()

loading model at checkpoints/ind_recommended


2024-03-06 14:55:01,253 - {pytorch_tabular.tabular_model:340} - INFO - Preparing the Trainer

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


predicting


Output()

In [45]:

gen_submission(idf)


generating_pred_score
missing col activate_pred
missing col rec_pred
formate prodiction


In [ ]:
result = {k: float(v) for k,v in result[0].items()}
result["mode"] = "Normal"

results.append(result)

## Custom Sampler

PyTorch Tabular also allows custom batching strategy through Custom Samplers  which comes in handy when working with imbalanced data.

Although you can use any sampler, Pytorch Tabular has a few handy utility functions which takes in the target array and implements WeightedRandomSampler using inverse frequency sampling to combat imbalance. This is analogous to preprocessing techniques like Under or OverSampling in traditional ML systems.

In [ ]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    verbose=False
)
sampler = get_balanced_sampler(train[target_col].values.ravel())

tabular_model.fit(train=train, validation=val, train_sampler=sampler)


In [ ]:
result = tabular_model.evaluate(test)

In [ ]:
result = {k: float(v) for k,v in result[0].items()}
result["mode"] = "Balanced Sampler"

results.append(result)

## Custom Weighted Loss

If Samplers were like Over/Under Sampling, Custom Weighted Loss is similar to `class_weights`. Depending on the problem, one of these might help you with imbalance. You can easily make calculate the class_weights and provide them to the CrossEntropyLoss using the parameter `weight`. To make this easier, PyTorch Tabular has a handy utility method which calculates smoothed class weights and initializes a weighted loss. Once you have that loss, it's just a matter of passing it to the 1fit1 method using the `loss` parameter.

In [ ]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    verbose=False
)
weighted_loss = get_class_weighted_cross_entropy(train["target"].values.ravel(), mu=0.1)

tabular_model.fit(train=train, validation=val, loss=weighted_loss)


In [ ]:
result = tabular_model.evaluate(test)

In [ ]:
result = {k: float(v) for k,v in result[0].items()}
result["mode"] = "Class Weights"

results.append(result)

In [ ]:
res_df = pd.DataFrame(results).T
res_df.columns = res_df.iloc[-1]
res_df = res_df.iloc[:-1].astype(float)
res_df.style.highlight_min(color="lightgreen",axis=1)